In [57]:
import os 
import numpy as np
import pandas as pd

import random
from PIL import Image,ImageDraw
from collections import Counter
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

import torch
import torchvision
import torchvision.transforms as T
from torchsummary import summary
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import gc


In [56]:
torch.cuda.empty_cache()
gc.collect()

6525

In [58]:
df =pd.read_csv('dataset.csv')
df.head()

,image_id,x1,y1,x2,y2
0,frame1001,7.66,92.77,80.85,109.79
1,frame10010,5.96,91.06,80.85,109.79
2,frame10087,7.66,91.92,82.55,110.64
3,frame10164,7.66,93.62,81.70,109.79
4,frame10241,8.51,92.77,84.26,109.79


In [59]:
unique_imgs=df.image_id.unique()


In [60]:
class CustDat(torch.utils.data.Dataset):
    def __init__(self,df,unique_imgs,indices):
        self.df=df
        self.unique_imgs=unique_imgs
        self.indices=indices
    def __len__(self):
        return len(self.indices)
    def __getitem__(self,idx):
        image_name = self.unique_imgs[self.indices[idx]]
        boxes=self.df[self.df.image_id==image_name].values[:,1:].astype("float")
        img = Image.open("./images/"+image_name+ ".jpg").convert('RGB')
        labels = torch.ones((boxes.shape[0]),dtype = torch.int64)
        target ={}
        target["boxes"] = torch.tensor(boxes)
        target["label"]= labels
        return T.ToTensor()(img),target

In [61]:
train_inds,val_inds = train_test_split(range(unique_imgs.shape[0]),test_size = 0.1)

In [62]:
def custom_collate(data):
   return data

In [63]:
train_dl = torch.utils.data.DataLoader(CustDat(df,unique_imgs,train_inds),batch_size = 16,shuffle=True,collate_fn = custom_collate,pin_memory= True if torch.cuda.is_available()else False)

val_dl = torch.utils.data.DataLoader(CustDat(df,unique_imgs,train_inds),batch_size = 8,shuffle=True,collate_fn = custom_collate,pin_memory= True if torch.cuda.is_available()else False)

In [64]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes=2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)

c:\Users\NDIN PC-01\anaconda3\envs\skdet\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\NDIN PC-01\anaconda3\envs\skdet\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [65]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [66]:
optimizer = torch.optim.SGD(model.parameters() , lr = 0.001,momentum=0.9,weight_decay=0.0005)
num_epochs= 100

In [67]:
model.to(device)
for epochs in range(num_epochs):
    epoch_loss = 0
    for data in train_dl:
        imgs=[]
        targets=[]
        for d in data:
            imgs.append(d[0].to(device))
            targ={}
            targ["boxes"] = d[1]["boxes"].to(device)
            targ["labels"] = d[1]["label"].to(device)
            targets.append(targ)
        loss_dict =model(imgs,targets)
        loss = sum(v for v in loss_dict.values())
        epoch_loss+=loss.cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(epoch_loss)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1008.00 MiB (GPU 0; 6.00 GiB total capacity; 3.96 GiB already allocated; 0 bytes free; 4.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.eval()
data =iter(val_d1).__next__()

In [ ]:
img =data[0][0]
boxes=data[0][1]["boxes"]
labels=data[0][1]["label"]

In [ ]:
output=model.([img.to(device)])

In [ ]:
out_bbox=output[0]["boxes"]
out_scores=output[0]["scores"]

In [ ]:
keep = torchvision.ops.nms(out_bbox,out_scores,0.45)

In [ ]:
out_bbox.shape,keep.shape

In [ ]:
im =(img.permute(1,2,0).cpu().detach().numpy()*255).astype('uint8')

In [ ]:
im

In [ ]:
draw=ImageDraw.Draw(vsample)
for box in boxes:
    draw.rectangle(list(box),fill=None,outline ="red")

vsample